In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## langchain.embeddings

In [3]:
# text = ""
# doc_embedding = embeddings.embed_documents(text)

from langchain.document_loaders import DirectoryLoader, TextLoader, CSVLoader
loader = CSVLoader('../unzip_data/a.csv')
documents = loader.load()

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
t = embeddings.embed_documents([t.page_content for t in documents])
print(len(t))
print(len(t[0]))
print(t[0][0])


2
768
0.042400114238262177


In [9]:
from langchain.vectorstores import Pinecone
import pinecone
from tqdm.auto import tqdm

index_name = "csv"

pinecone.init(
    api_key="6664c379-ef4f-41fb-afc4-7242589d2c07",
    environment="asia-southeast1-gcp-free"
)

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=768)
# else:
#     pinecone.delete_index(index_name)
#     pass

index = pinecone.Index(index_name)
# docsearch = Pinecone.from_texts([t.page_content for t in tqdm(documents)], embeddings, index_name=index_name)

In [10]:
batch_size = 32
for i in tqdm(range(0, len(documents), batch_size)):
    i_end = min(i+batch_size, len(documents))
    vector_id = [str(j) for j in range(i, i_end)]
    embedding_words = embeddings.embed_documents([t.page_content for t in documents[i:i_end]])
    texts = [{'text': t.page_content} for t in documents[i:i_end]]
    to_upsert = zip(vector_id, embedding_words, texts)
    index.upsert(vectors=list(to_upsert))


    # docsearch = Pinecone.from_texts([t.page_content for t in tqdm(documents[i:i+batch_size])], embeddings, index_name=index_name)

  0%|          | 0/1843 [00:00<?, ?it/s]

In [11]:
query = "what is the DISCHTIME going to be when his/her ADMITTIME is 2157-10-18 19:34:00 and its DIAGNOSIS is CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA?"
query = "what is the DISCHTIME going to be when his/her ADMITTIME is 2153-09-03 07:15:00 and its DIAGNOSIS is CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA?"

find_index = index.query(queries=[embeddings.embed_documents([query])], top_k=15, include_metadata=True)
'''
res -> results -> [0]
    -> matches -> [id, metadata, score]
        -> metadata -> text
'''


'\nres -> results -> [0]\n    -> matches -> [id, metadata, score]\n        -> metadata -> text\n'

In [12]:
find_text = [r['metadata']['text'] for r in find_index['results'][0]['matches']]
# for i in range(len(find_text)):
#     print(find_text[i].split('\n')[16])

In [13]:
# prompt = f"From all the csv file,some patient information is as follows:{find_text}. Could you brainstorm three distinct solutions? Please consider a variety of factors.three different experts are answering this question. All experts will write down 1 step of their thinking,then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point then they leave. The question is {query}"

prompt = f"From all the csv file,some patient information is as follows:{find_text}. I have a problem related to {query}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as ADMITTIME, DIAGNOSIS and DISCHTIME.For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors.Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution."

In [14]:
import openai
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.2,
)
print(res['choices'][0].message)

ServiceUnavailableError: The server is overloaded or not ready yet.

## langchain.agents

In [ ]:
from langchain import OpenAI
from langchain.agents import create_csv_agent

path = "unzip_data/ADMISSIONS.csv"
llm = OpenAI(temperature=0)
agent = create_csv_agent(llm, path, verbose=True)
problem = "from all the csv file, what is the DISCHTIME going to be when his/her ADMITTIME is 2157-10-18 19:34:00 and DIAGNOSIS is INTERIOR MYOCARDIAL INFARCTION?"
prompt = '''
I have a problem related to ''' + problem + '''. Could you brainstorm three distinct solutions? Please consider a variety of factors.
For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors.
For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.
Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution.
'''
print(prompt)
agent.run(prompt)

In [ ]:
problem = "from all the csv file, you need to predict what is the DISCHTIME going to be when his/her ADMITTIME is 2157-10-18 19:34:00 and DIAGNOSIS is INTERIOR MYOCARDIAL INFARCTION?"
prompt = '''
I have a problem related to ''' + problem + '''. Could you brainstorm three distinct solutions? Please consider a variety of factors.
For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors.
For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.
Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution.
'''
print(prompt)
agent.run(prompt)